In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Мне больше нравится plotly поэтому я использую ее вместо matplotlib

In [17]:
train_data = pd.read_csv('in/train.csv')
test_data = pd.read_csv('in/test.csv')

In [18]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Проверим сколько у нас пропущенных значений

In [20]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Посмотрим сколько выжило

In [42]:
check_count_survived = train_data.groupby(by=['Survived']).size().reset_index()

In [43]:
check_count_survived

,Survived,0
0,0,549
1,1,342


In [69]:
go.Figure(
    data=[
        go.Pie(
            labels=check_count_survived['Survived'],
            values=check_count_survived[0],
            textinfo='label+value+percent'
        )
    ]
)

Проверим зависимость выживания от пола

In [64]:
check_count_survived_sex = train_data.groupby(by=['Survived', 'Sex']).size().reset_index()

In [65]:
check_count_survived_sex

,Survived,Sex,0
0,0,female,81
1,0,male,468
2,1,female,233
3,1,male,109


In [74]:
go.Figure(
    go.Bar(
        x=check_count_survived_sex['Survived'],
        y=check_count_survived_sex[0],
        text=check_count_survived_sex['Sex'],
        textposition='auto'
    )
)

Далее взглянем, как зависела выживаемость от класса каюты.

In [75]:
check_count_survived_pclass = train_data.groupby(by=['Survived', 'Pclass']).size().reset_index()

In [76]:
check_count_survived_pclass

,Survived,Pclass,0
0,0,1,80
1,0,2,97
2,0,3,372
3,1,1,136
4,1,2,87
5,1,3,119


In [77]:
go.Figure(
    go.Bar(
        x=check_count_survived_pclass['Survived'],
        y=check_count_survived_pclass[0],
        text=check_count_survived_pclass['Pclass'],
        textposition='auto'
    )
)

В качестве признаков кроме выбранных пола и класса каюты, возьмем количество родителей детей  и количество братьевсестер на борту.

In [78]:
features = ['Sex', 'Pclass', 'SibSp', 'Parch']

Информацию о выживших и погибших пассажирах поместим в переменную y:

In [79]:
y = train_data['Survived']

Теперь заменим текстовые поля с полом на числа

In [80]:
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [81]:
X.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,1,0,0,1
1,1,1,0,1,0
2,3,0,0,1,0
3,1,1,0,1,0
4,3,0,0,0,1


Теперь самое интересное))
Попробуем обучить нашу модель методом решающих деревьев.

In [82]:
from sklearn.ensemble import RandomForestClassifier

In [83]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)  # обучаем модель
prediction = model.predict(X_test)  # делаем предсказание
output = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':prediction})
output.to_csv('Out/my_submission.csv', index=False)  # формируем итоговый датафрейм и сохраняем его в csv файл